# What

As part of the produciton challenge #172. I need to be able to upload the results to an external server.


## Modules

In [ ]:
# Local

# Third Party
import lancedb
import pandas as pd

# Built in


# Upload output to viewer vector_db
The end goal will be to take the embeddings datasets and put them into a vector database that is in the cloud.

However right not I am not working on it so I can just create this local script that will take the embedding folder and move it into a vector_db database within the viewer.

In [ ]:
db = lancedb.connect("../../viewer/vector_db")

In [ ]:
important_text_embeddings = pd.read_pickle("../../output/embeddings/important_text_embeddings.pkl")

important_text_embeddings.rename(columns={'important_text_embedding': 'vector'}, inplace=True)
db.create_table("important_text_embeddings", important_text_embeddings, mode="overwrite")


In [ ]:
recommendations_embeddings = pd.read_pickle("../../output/embeddings/recommendations_embeddings.pkl")

recommendations_embeddings.rename(columns={'recommendation_embedding': 'vector'}, inplace=True)
db.create_table("recommendation_embeddings", recommendations_embeddings, mode="overwrite")

In [ ]:
report_sections_embeddings = pd.read_pickle("../../output/embeddings/report_sections_embeddings.pkl")

report_sections_embeddings.rename(columns={'section_embedding': 'vector'}, inplace=True)
section_table = db.create_table("report_section_embeddings", report_sections_embeddings, mode="overwrite")

section_table.create_fts_index("section_text", replace=True)

In [ ]:
safety_issues_embeddings = pd.read_pickle("../../output/embeddings/safety_issues_embeddings.pkl")

safety_issues_embeddings.rename(columns={'safety_issue_embeddings': 'vector'}, inplace=True)
si_table = db.create_table("safety_issue_embeddings", safety_issues_embeddings, mode="overwrite")

si_table.to_pandas()